### Dedupe Example from datamade

Steps
1. pre-process data (strip unicode characters, whitspace,newlines,",', make everython lowercase
2. read in data from csv file and convert to python dictionary 
3. train model (active modeling by user) 
4. 

In [7]:
from future.builtins import next

import os
import csv
import re
import collections
import logging
import optparse
from numpy import nan
import pandas as pd
import dedupe
from unidecode import unidecode

In [2]:
#
optp = optparse.OptionParser()
optp.add_option('-v', '--verbose', dest='verbose', action='count',
                help='Increase verbosity (specify multiple times for more)'
                )
(opts, args) = optp.parse_args()
log_level = logging.WARNING 
if opts.verbose :
    if opts.verbose == 1:
        log_level = logging.INFO
    elif opts.verbose >= 2:
        log_level = logging.DEBUG
logging.getLogger().setLevel(log_level)
#

Usage: __main__.py [options]

__main__.py: error: no such option: -f


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [3]:
input_file = 'csv_example_messy_input.csv'
output_file = 'csv_example_output.csv'
settings_file = 'csv_example_learned_settings'
training_file = 'csv_example_training.json'

In [4]:
def preProcess(column):
    #For each column in csv file strip unicode characters, whitespace, newlines, apostrophe, lowercase all text 
    import unidecode
    column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column

In [5]:
def readData(filename):
# read in data from csv file, apply preprocess function, return a dictionary (row id: {cleaned data  site name:xx, adress: xxx})
    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

In [8]:
# example of messy input data
data = pd.read_csv(input_file)
data.head()

,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,Length of Day,IDHS Provider ID,...,Executive Director,Center Director,ECE Available Programs,NAEYC Valid Until,NAEYC Program Id,Email Address,Ounce of Prevention Description,Purple binder service type,Column,Column2
0,0,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CPS_Early_Childhood_Portal_scrape.csv,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave,NaN,2916100,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#example of dictionary 
print data_d[10]

{'Purple binder service type': None, 'Zip': None, 'Site name': 'hull house association - child dev. central office- hull house cds-homes', 'Program Name': 'child care', 'Agency': None, 'Source': 'cps_early_childhood_portal_scrape.csv', 'Eearly Head Start Fund': None, 'Length of Day': 'extended day', 'NAEYC Program Id': None, 'Website': None, 'Program Option': None, 'Fax': None, 'Head Start Fund': None, 'Ounce of Prevention Description': None, 'NAEYC Valid Until': None, 'Director': None, 'Phone': '9068600', 'Address': '1030 w van buren st', 'ECE Available Programs': None, 'CC fund': None, 'Email Address': None, 'Column': None, 'Center Director': None, 'Funded Enrollment': None, 'Column2': None, 'Number per Site EHS': None, 'Neighborhood': None, 'Progmod': None, 'IDHS Provider ID': None, 'Number per Site HS': None, 'Executive Director': None, 'Id': '10'}


In [11]:
print('importing data ...')
data_d = readData(input_file)

## Training

if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)

else:
    # Define the fields dedupe will pay attention to
    #
    # Notice how we are telling dedupe to use a custom field comparator
    # for the 'Zip' field. 
    fields = [
        {'field' : 'Site name', 'type': 'String'},
        {'field' : 'Address', 'type': 'String'},
        {'field' : 'Zip', 'type': 'Exact', 'has missing' : True},
        {'field' : 'Phone', 'type': 'String', 'has missing' : True},
        ]

    # Create a new deduper object and pass our data model to it.
    deduper = dedupe.Dedupe(fields)

    # To train dedupe, we feed it a sample of records.
    deduper.sample(data_d, 15000)


    # If we have training data saved from a previous run of dedupe,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.readTraining(f)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as duplicates
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.consoleLabel(deduper)

    deduper.train()

    # When finished, save our training away to disk
    with open(training_file, 'w') as tf :
        deduper.writeTraining(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf :
        deduper.writeSettings(sf)


importing data ...
starting active labeling...

Phone : None
Address : 2629 s lawndale
Zip : 60623
Site name : chicago commons association a-z mall world

Phone : 77352211561150
Address : 2629 s lawndale
Zip : 60623
Site name : chicago commons association a-z mall world

Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished



y


Phone : 3813652
Address : 7638 n paulina ave
Zip : None
Site name : howard area community center - howard area

Phone : 9787827
Address : 3012-16 e. 92nd st.
Zip : None
Site name : shinning star child development institute

Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


...

Phone : None
Address : 2718 w 59th st
Zip : None
Site name : easter seals society of metropolitan chicago - the keeper's inst.

Phone : 9181944
Address : 3038 w 59th street
Zip : 60629
Site name : el valor young scholars dev. ins.

Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


Phone : 3757022
Address : 10530 s oglesby
Zip : None
Site name : ada s. mckinley community services - mckinley - trumbull park

Phone : 9785341
Address : 10530 s oglesby
Zip : 60617
Site name : ada s. mckinley community services mckinley-trumbull park

Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


f


Finished labeling


In [12]:
# ## Blocking

print('blocking...')

# ## Clustering

# Find the threshold that will maximize a weighted average of our precision and recall. 
# When we set the recall weight to 2, we are saying we care twice as much
# about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

threshold = deduper.threshold(data_d, recall_weight=1)

# `match` will return sets of record IDs that dedupe
# believes are all referring to the same entity.

print('clustering...')
clustered_dupes = deduper.match(data_d, threshold)

print('# duplicate sets', len(clustered_dupes))

# ## Writing Results

# Write our original data back out to a CSV with a new column called 
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores) :
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

singleton_id = cluster_id + 1

with open(output_file, 'w') as f_output:
    writer = csv.writer(f_output)

    with open(input_file) as f_input :
        reader = csv.reader(f_input)

        heading_row = next(reader)
        heading_row.insert(0, 'confidence_score')
        heading_row.insert(0, 'Cluster ID')
        canonical_keys = canonical_rep.keys()
        for key in canonical_keys:
            heading_row.append('canonical_' + key)
        
        writer.writerow(heading_row)

        for row in reader:
            row_id = int(row[0])
            if row_id in cluster_membership :
                cluster_id = cluster_membership[row_id]["cluster id"]
                canonical_rep = cluster_membership[row_id]["canonical representation"]
                row.insert(0, cluster_membership[row_id]['confidence'])
                row.insert(0, cluster_id)
                for key in canonical_keys:
                    row.append(canonical_rep[key].encode('utf8'))
            else:
                row.insert(0, None)
                row.insert(0, singleton_id)
                singleton_id += 1
                for key in canonical_keys:
                    row.append(None)
            writer.writerow(row)

blocking...
clustering...
('# duplicate sets', 809)


###Sample output file 

Clusters: blocks of potential duplicate records 
Confidence_score: probability that the records in cluster match

In [9]:
output = pd.read_csv(output_file)
output.head()

,Cluster ID,confidence_score,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,...,canonical_Center Director,canonical_Funded Enrollment,canonical_Column2,canonical_Number per Site EHS,canonical_Neighborhood,canonical_Progmod,canonical_IDHS Provider ID,canonical_Number per Site HS,canonical_Executive Director,canonical_Id
0,0,0.880895,0,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1
1,0,0.880895,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1
2,1,0.739643,2,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011,NaN,Child Care,...,www.ymcachicago.org,ymca of metropolitan chicago,NaN,NaN,east garfield park,no,NaN,NaN,4364ps(hs collaboration with childcare & prek),2
3,1,0.739643,3,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011,NaN,Child Care,...,www.ymcachicago.org,ymca of metropolitan chicago,NaN,NaN,east garfield park,no,NaN,NaN,4364ps(hs collaboration with childcare & prek),2
4,2,0.944684,4,CPS_Early_Childhood_Portal_scrape.csv,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave,NaN,2916100,NaN,Child Care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513
